In [176]:
import pandas as pd
import numpy as np

from subprocess import call

from scipy.stats import pearsonr as pcor

## Functional genomic analysis
Here we will analyse the cell death associated signature from functional genomic aspect.

At first we will calculate gene expression - cell viability correlation coefficients form Achilles-L1000-96h (and also CTRP-L1000-24h) datasets.

In [34]:
#achilles data
sig_info=pd.read_table('../results/Achilles/sig_info_merged_lm.csv',
                      sep=',',header=0,index_col=[0])
signatures=pd.read_table('../results/Achilles/signatures_merged_lm.csv',
                        sep=',',header=0,index_col=[0])
fil=sig_info['pert_itime']=='96 h'
sig_info=sig_info[fil]
signatures=signatures.loc[sig_info.index]

In [35]:
#prepare a dataframe to store correlations
gene_info=pd.read_table('../data/LINCS/GSE92742/GSE92742_Broad_LINCS_gene_info.txt',
                       sep='\t',index_col=[0],header=0)
fil=gene_info['pr_is_lm']==1
gene_info=gene_info[fil]
gene_info.index=gene_info.index.astype(str)
correlations=pd.DataFrame(index=gene_info.index,
                           columns=['pr_gene_symbol','Pearson_r','p_val'])
correlations['pr_gene_symbol']=gene_info['pr_gene_symbol']

In [36]:
for gene_id in correlations.index:
    r,p=pcor(signatures[gene_id],sig_info['shRNA_abundance'])
    correlations.loc[gene_id,['Pearson_r','p_val']]=r,p
correlations.to_csv('../results/functional/achilles_cors_lm.csv')

In [37]:
#just looking into the gene - viaiblity correlations
correlations.sort_values('Pearson_r',ascending=False).head()

,pr_gene_symbol,Pearson_r,p_val
pr_gene_id,,,
890,CCNA2,0.342786,0
22974,TPX2,0.336666,0
55723,ASF1B,0.332808,0
332,BIRC5,0.328564,0
10460,TACC3,0.326702,0


In [38]:
# do the same analysis with ctrp
sig_info=pd.read_table('../results/CTRP/sig_info_merged_lm.csv',
                      sep=',',header=0,index_col=[0])
signatures=pd.read_table('../results/CTRP/signatures_merged_lm.csv',
                        sep=',',header=0,index_col=[0])
fil=sig_info['pert_itime']=='24 h'
sig_info=sig_info[fil]
signatures=signatures.loc[sig_info.index]
correlations=pd.DataFrame(index=gene_info.index,
                           columns=['pr_gene_symbol','Pearson_r','p_val'])
correlations['pr_gene_symbol']=gene_info['pr_gene_symbol']
for gene_id in correlations.index:
    r,p=pcor(signatures[gene_id],sig_info['cpd_avg_pv'])
    correlations.loc[gene_id,['Pearson_r','p_val']]=r,p
correlations.to_csv('../results/functional/ctrp_cors_lm.csv')

## Gene Ontology Enrichment

In [177]:
# Gene Ontology Enrichments
call(['Rscript','GO_BP_enrichments.R'])

0

In [185]:
#just look into GO results
data=pd.read_table('../results/functional/enrichments/GOBP_achilles.tsv',
                   sep='\t',header=0,index_col=[0])
data.sort_values('p adj (dist.dir.up)').head(10)

,Name,Genes (tot),Stat (dist.dir),p (dist.dir.up),p adj (dist.dir.up),p (dist.dir.dn),p adj (dist.dir.dn),Genes (up),Genes (down)
633,GO_MITOTIC_CELL_CYCLE,125,0.49443,0.000216,0.007074,NaN,NaN,80,45
267,GO_COENZYME_METABOLIC_PROCESS,27,0.59217,0.000210,0.007074,NaN,NaN,25,2
638,GO_MITOTIC_NUCLEAR_DIVISION,59,0.61177,0.000210,0.007074,NaN,NaN,40,19
326,GO_DNA_REPLICATION,30,0.68915,0.000211,0.007074,NaN,NaN,22,8
2019,GO_TRANSCRIPTION_COUPLED_NUCLEOTIDE_EXCISION_R...,16,0.71986,0.000208,0.007074,NaN,NaN,13,3
325,GO_DNA_REPAIR,52,0.52205,0.000208,0.007074,NaN,NaN,38,14
985,GO_PEPTIDYL_LYSINE_MODIFICATION,35,0.60811,0.000212,0.007074,NaN,NaN,22,13
324,GO_DNA_RECOMBINATION,20,0.75075,0.000211,0.007074,NaN,NaN,16,4
316,GO_DNA_DEPENDENT_DNA_REPLICATION,18,0.71281,0.000211,0.007074,NaN,NaN,13,5
639,GO_MITOTIC_RECOMBINATION,11,0.85123,0.000205,0.007074,NaN,NaN,10,1


## KEGG pathway enrichment

In [186]:
# KEGG pathway Enrichments
call(['Rscript','KEGG_enrichments.R'])

0

In [259]:
data=pd.read_table('../results/functional/enrichments/KEGG_achilles.tsv',
                   sep='\t',header=0,index_col=[0])
data.sort_values('p adj (dist.dir.up)').head(10)

,Name,Genes (tot),Stat (dist.dir),p (dist.dir.up),p adj (dist.dir.up),p (dist.dir.dn),p adj (dist.dir.dn),Genes (up),Genes (down)
67,KEGG_NUCLEOTIDE_EXCISION_REPAIR,13,0.74681,0.000205,0.005046,NaN,NaN,11,2
60,KEGG_MISMATCH_REPAIR,9,0.84135,0.000408,0.005046,NaN,NaN,8,1
26,KEGG_DNA_REPLICATION,10,0.85379,0.000409,0.005046,NaN,NaN,9,1
12,KEGG_BASE_EXCISION_REPAIR,7,0.84383,0.000610,0.005639,NaN,NaN,5,2
18,KEGG_CELL_CYCLE,42,0.46980,0.000817,0.006045,NaN,NaN,27,15
91,KEGG_SPLICEOSOME,11,0.70556,0.002450,0.015108,NaN,NaN,9,2
83,KEGG_PURINE_METABOLISM,12,0.67100,0.003066,0.016204,NaN,NaN,8,4
84,KEGG_PYRIMIDINE_METABOLISM,10,0.66115,0.011661,0.053933,NaN,NaN,7,3
68,KEGG_OOCYTE_MEIOSIS,18,0.52426,0.021351,0.087776,NaN,NaN,11,7
89,KEGG_RNA_DEGRADATION,8,0.62389,0.061738,0.190360,NaN,NaN,6,2


## Transcription factor regulon enrichment
Regulons for TFs are already included in the repository, but can manually dowloaded from [DoRothEA repository](https://github.com/saezlab/DoRothEA).

In [250]:
# Transcription Factor regulon enrichment using DoRothEA and viper
call(['Rscript','TF_regulon_enrichments.R'])

0

In [251]:
# we remove low confidence level TFs from enrichment results
data=pd.read_table('../results/functional/enrichments/DoRothEA.csv',
                  sep=',',header=0,index_col=[0])
tf_names=pd.Series(data.index).apply(lambda x:x.split('_')[0]).values
good_msk=pd.Series(data.index).apply(lambda x:x.split('_')[1][0] in ['A','B']).values
data.index=tf_names
data=data[good_msk]
data.columns=['NES']

In [252]:
data.to_csv('../results/functional/enrichments/DoRothEA.csv')
print(data.sort_values('NES'))

             NES
FOXO3  -3.816854
PRDM14 -3.440174
ESR2   -2.888764
POU2F1 -2.772940
TP53   -2.701049
USF2   -2.554411
STAT1  -2.442873
FOS    -2.313918
ETS1   -2.248835
JUN    -2.153735
ESR1   -1.969761
SPI1   -1.950526
SMAD4  -1.942263
TWIST1 -1.937114
FOXO1  -1.922666
SMAD3  -1.901023
FOXO4  -1.836478
TFAP2A -1.746101
ETV4   -1.743225
FLI1   -1.582792
SOX2   -1.581124
ATF2   -1.571681
AR     -1.498928
KLF4   -1.479860
USF1   -1.413601
VDR    -1.411384
HIF1A  -1.381960
PAX8   -1.331544
CTCF   -1.326600
ATF4   -1.325524
...          ...
CREB1  -0.429856
MYB    -0.287150
FOXL2  -0.286375
SREBF1 -0.276869
RELA   -0.213521
STAT3  -0.149852
WT1    -0.081892
RARA   -0.008986
FOXP1   0.014709
ELK1    0.103107
SP1     0.127504
CEBPD   0.134245
HNF4A   0.218703
ZNF263  0.450042
STAT5A  0.451467
TFAP2C  0.461781
ETS2    0.795876
E2F3    1.058977
TCF7L2  1.080351
FOXA1   1.217310
YY1     1.519722
SREBF2  1.527549
BACH1   1.759120
MYC     1.821395
ATF1    1.863959
FOXM1   2.468741
TFDP1   3.0689

## Signaling pathway footrpint analysis (PROGENy)
Model matrix for PROGENy is already included in the repository, but can manually dowloaded from [PROGENy repository](https://github.com/saezlab/progeny)

In [253]:
# for PROGENy we have low overlap between landmark and PROGENY genes
# so we use bing gene - cell viability correlations
# to calculate this, you have to re-run a first part of this
# notebook with bing genes: replacing fil=gene_info['pr_is_lm']==1
# to fil=gene_info['pr_is_bing']==1 in second cell (and renaming output files *_bing.*)
correlations=pd.read_table('../results/functional/achilles_cors_bing.csv',
                          sep=',',header=0,index_col=[0])
progeny=pd.read_table('../data/Functional/PROGENy.csv',
                     sep=',',header=0,index_col=[0])
correlations.index=correlations['pr_gene_symbol']
correlations=pd.DataFrame(correlations['Pearson_r'])

In [254]:
genes=list(set(progeny.index)&set(correlations.index))
print(len(genes))

898


In [255]:
scores=pd.DataFrame(index=range(1000),columns=progeny.columns)
#first real pathway activity
scores.loc[0]=np.dot(correlations.loc[genes].T,progeny.loc[genes])
# than 1000 random permutation of genes in correlations
np.random.seed(19890904)
for i in range(1,1000):
    correlations.index=np.random.choice(correlations.index,len(correlations.index),False)
    scores.loc[i]=np.dot(correlations.loc[genes].T,progeny.loc[genes])

In [256]:
scores=((scores-np.mean(scores,0))/np.std(scores,0)).loc[0]
scores=pd.DataFrame(scores)
scores.columns=['Pathway activity']

In [257]:
scores.to_csv('../results/functional/enrichments/PROGENy.csv')
print(scores)

         Pathway activity
EGFR              2.93174
Hypoxia           2.29129
JAK-STAT         -3.24846
MAPK              4.29581
NFkB             -1.26311
PI3K              2.90661
TGFb              -1.3792
TNFa             -1.18241
Trail            0.912791
VEGF             0.330147
p53              -1.99183
Androgen           1.5352
Estrogen          2.35669
WNT              -1.75701


## Association with drug sensitivity
We will calculte the "cell viability" score for the cancer cell lines in the GDSC panel, and see that this score is associated with drug sensitivity. 

In [260]:
# translating gene ids
call(['Rscript','gene_translate.R'])

0

In [286]:
expression=pd.read_table('../data/GDSC/sanger1018_brainarray_ensemblgene_rma.txt',
                        sep='\t',header=0,index_col=[0])
gene_anno=pd.read_table('../data/GDSC/ensembl_hgnc.csv',sep=',',header=0,index_col=[0])
# remove not 1on1 translations and NaNs
gene_anno=gene_anno.drop_duplicates('ensembl_gene_id')
gene_anno=gene_anno.drop_duplicates('hgnc_symbol')
msk=np.sum(pd.isnull(gene_anno),1)==0
gene_anno=gene_anno[msk]
# translate gdsc gene ids gene symbol
gene_anno.index=gene_anno['ensembl_gene_id']
gene_anno=gene_anno['hgnc_symbol']
genes=list(set(gene_anno.index)&set(expression.index))
expression=expression.loc[genes]
gene_anno=gene_anno[genes]
expression.index=gene_anno.values
expression.to_csv('../data/GDSC/gex.csv',sep=',')
# also normalising the gene expression
expression_norm=((expression.T-np.mean(expression,1))/np.std(expression,1)).T
expression_norm.to_csv('../data/GDSC/gex_norm.csv',sep=',')

In [294]:
# now let's calculate "cell viability signature score"
model=pd.read_table('../results/model/final_models/achilles.csv',sep=',',
                   header=0,index_col=[0])
model.index=model['pr_gene_symbol']
model=pd.DataFrame(model['coefficient'])
genes=list(set(model.index) & set(expression_norm.index))
print('We have %i common genes between cancer cell lines and LINCS models.' % len(genes))

We have 907 common genes between cancer cell lines and LINCS models.


In [304]:
score=pd.DataFrame(np.dot(expression_norm.loc[genes].T,model.loc[genes]),
             index=expression_norm.columns,columns=['Signature_score'])
score.index.name='COSMIC'
score.to_csv('../results/functional/gdsc_scores.csv',sep=',')

We will compare this signature score with different other things, like cell division rate, general drug sensitivity and drug specific sensitivity. For cell division rate, we need to download **DS1.zip** from [Hafner et al, Scientific Data 2017](https://datadryad.org/resource/doi:10.5061/dryad.03n60). **DS1_datafile.xls** goes to *'../data/GDSC/*.

In [333]:
growth=pd.read_excel('../data/GDSC/DS1_datafile.xlsx')
growth=growth.drop_duplicates('Cell Name')
growth=growth[['Cell Name','Nominal Division Rate']]
growth.index=growth['Cell Name']
growth=growth['Nominal Division Rate']
#we get COSMIC IDs for these cell lines
cell_anno=pd.read_excel('../data/GDSC/Cell_Lines_Details.xlsx')
cell_anno.index=cell_anno['Sample Name']
cells=list(set(growth.index)&set(cell_anno.index))
growth=growth[cells]
cell_anno=cell_anno.loc[cells]
growth.index=cell_anno['COSMIC identifier'].astype(int).astype(str)